<a href="https://colab.research.google.com/github/RMeuter/NLPHumourProjet/blob/ironie/ironie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and dependencies

## Téléchargement / Imports des librarys et des fichiers

Installation :


In [1]:
!pip3 install langdetect
!pip3 install stanza
!pip3 install nltk
!pip3 install pygrammalecte

Imports :


In [2]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
import stanza
from stanza.pipeline.processor import Processor, register_processor
from langdetect import *
#from custom_process import *
from collections import Counter
import nltk

from pathlib import Path
from pygrammalecte import*
# ATTENTION necessite le fichier mots_fr.py (dico mots français) et dico.py (dico verbes et conjugaions français)

from google.colab import drive
drive.mount('/content/drive')

PATH = "/content/drive/MyDrive/Colab Notebooks/colab_SMS/"
# PATH = "content/drive/MyDrive/Colab Notebooks/SMS/"
sys.path.append(PATH)
import correcteur as cor
# ATTENTION necessite le fichier corrector.py

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load / Download Stanzas and NLTK dependencies

In [3]:
stanza.download("fr") 
nltk.download("stopwords")

2021-04-30 09:01:14 INFO: Downloading default packages for language: fr (French)...
2021-04-30 09:01:16 INFO: File exists: /root/stanza_resources/fr/default.zip.
2021-04-30 09:01:23 INFO: Finished downloading models and saved to /root/stanza_resources.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Corpus Traitment

## Notre corpus de sms / Source : 88milSMS_88522_formated.csv

Dataframe :

In [4]:
FILENAME = PATH + "SMS/88milSMS_88522_formated.csv" # "data/88milSMS_88522_formated.csv"

dfSms = pd.read_csv(FILENAME, sep=',')
# dfSms = dfSms.rename(columns={dfSms.columns[0] : "Index"}).set_index(dfSms.columns[0])
# dfSms = dfSms.set_index(dfSms.cols[0])

dfSms.head()

,Unnamed: 0,NUM_SMS,DATETIME,ID_NUM_TEL,sms,langue
0,0,1,15 sept. 2011 07:28:55,477,Hey ca va?\nAlors cette rentree?\nVa falloir s...,fr
1,1,2,15 sept. 2011 08:02:08,477,Ok super merci! Oui j'y comprends rien du tout...,fr
2,2,3,15 sept. 2011 08:03:01,477,Coucou !\nC'est quand la feria de Nimes? \nJ's...,fr
3,3,4,15 sept. 2011 08:03:35,477,Coucou :)\nOui ca c'est bien passe!\nAlors je ...,fr
4,4,5,15 sept. 2011 09:05:28,477,On peut se rejoindre quelque part? Tu as cours...,fr


## Dico python contenant nos balises en clé et leurs remplaçants


In [5]:
listePRE = {
    "PRE_" : "prénom",
    "SUR_" : "surnom",
    "TEL_" : "téléphone",
    "LIE_" : "lieu",
    "NOM_" : "nom",
}

listePRE

{'LIE_': 'lieu',
 'NOM_': 'nom',
 'PRE_': 'prénom',
 'SUR_': 'surnom',
 'TEL_': 'téléphone'}

## Dico émotions FEEL / Source : FEEL.csv

In [6]:
# Liste des colonnes de Feel qui nous interesse pour notre calcul de le poids des émotions d'un mot
feelings_col = ["joy", "fear", "sadness", "anger", "surprise", "disgust"]
dFeel = pd.read_csv(PATH + "SMS/FEEL.csv", sep=';')

# Ajout d'une colonne 'poids' dans la matrice dFeel contenant le poids des émotions d'un mot
dFeel['poids'] = dFeel.apply(lambda x : sum(x[feelings_col]) * -1 if x['polarity'] == "negative" else 1, axis = 1)

dFeel.head()

,id,word,polarity,joy,fear,sadness,anger,surprise,disgust,poids
0,1,à ce endroit là,positive,0,0,0,0,0,0,1
1,2,à le hâte,negative,0,1,0,0,1,0,-2
2,3,à part,negative,0,0,1,0,0,0,-1
3,4,à pic,negative,0,1,0,0,0,0,-1
4,5,à rallonge,negative,0,0,1,0,0,0,-1


# Pipeline Stanza

## Attribution de poids associé à l'émotion que le mots possède


In [7]:
# Fonction emotion qui retourne un dictionnaire contenant le poids de chaque mot d'un texte cible
def emotion(text):
        sentiment = {}
        for w in text.split(" "):                    
            for i in range(dFeel.shape[0]):
                elem = dFeel.iloc[i, :]
                if w.find(elem.word) != -1:
                    sentiment[w]= elem.poids
        return sentiment

# Processeur feel qui ajoute une propriétés feel_weight qui attribut à chaque mot un poids d'émotion (qui est la somme des différentes colonnes émotions de notre dFeel)
@register_processor("feel")
class FeelProcessor(Processor):
    _requires = set(["tokenize","mwt"])
    _provides = set(["feel"])

    def __init__(self, config, pipeline, use_gpu):
        pass

    def _set_up_model(self, *args):
        pass
    
    def process(self, doc):
        feel_fr={}
        for i,sent in enumerate(doc.sentences):
            feel_fr[i]=emotion(sent.text)
        try:
            doc.add_property('feel_weight', default={}, getter=lambda self: feel_fr, setter=None)
        except:
            pass

        return doc

## Definit le type de la phrase

On regarde la ponctuation afin de déterminer le type de phrase employé.
'?' montre que l'on a une phrase interrogativee, '!' une phrase exclamative .. et ainsi de suite .

In [8]:
# Processeur sent_type qui ajoute une propriétés sent_type qui attribut a chaque phrase son type phrase (interrogative,exclamative,..)
@register_processor("sent_type")
class SentenceTypeProcessor(Processor):
    ''' Processor that define the type of the sentence '''
    _requires = set(["tokenize"])
    _provides = set(["sent_type"])

    def __init__(self, config, pipeline, use_gpu):
        self.specific_punctuation = { # (sentence_type, weight)
            '?' : ("interrogative",0),
            '!' : ("exclamative",0),
            '.' : ("declarative",0),
            "..." : ("attente",0),
            ".." : ("attente",0),
            "?!" : ("exclamative-interrogative",0),
            "!?" : ("exclamative-interrogative",0)
        }
    
    def _set_up_model(self, *args):
        pass

    def _type_aux(self, sent):
        t = None
        for w in sent.words:
            for punc in self.specific_punctuation:
                index = w.text.find(punc)
                if index != -1:
                    t = self.specific_punctuation[punc]
        return t

    def process(self, doc):
        sent_type = {}
        for i, sent in enumerate(doc.sentences):
            key = "sent_{}".format(i+1)
            sent_type[key] = self._type_aux(sent)
        try:
            doc.add_property("sent_type", default={}, getter=lambda self: sent_type, setter=None)
        except:
            pass
        return doc

## Correction orthographique et grammaticale


### Correction à l'aide de pygrammalecte

[Github](https://github.com/vpoulailleau/pygrammalecte)

In [9]:
def handle_grammatical_error(sent, message):
    ''' Return a list of changes to do '''
    change = {}
    err = sent[message.start:]
    if message.type == "nbsp":
        change[err] = message.suggestions
    elif message.type == "imp":
        change[err] = message.suggestions
    return change

def grammalecte_corrector(sms):
    changes = {}
    for message in grammalecte_text(sms):
        # if type(message) == pygrammalecte.pygrammalecte.GrammalecteSpellingMessage:
        #     correction = _handle_spelling_error(sent.text, message)
        #     changes["orth"].update(correction)
        if type(message) == pygrammalecte.GrammalecteGrammarMessage:
            correction = handle_grammatical_error(sms, message)
            changes.update(correction)
        
    for err, remp in changes.items():
        if sms.find(err) != -1:
            sms = sms.replace(err, remp[0])

    return sms


@register_processor("grammalecte")
class GrammalecteProcessor(Processor):
    _requires = set(["tokenize"])
    _provides = set(["grammalecte"])

    def __init__(self, config, pipeline, use_gpu):
        pass
    
    def _set_up_model(self, *args):
        pass

    def _handle_spelling_error(selfsent, message):
        ''' Return a list of changes to do '''
        change = {}
        pro = cor.propositions(cor.arbre, message.word, 1)
        if pro != {}:
            done = False
            for remp in pro.keys():
                if remp.find("é") != -1:
                    change[message.word] = remp
                    done = True
                    # res = sent.replace(message.word, remp)
            if not done:
                change[message.word] = list(pro.keys())[1]

        for _, remp in change.items():
            erreurs = cor.verification(remp)
            for err in erreurs:
                pro = list(cor.propositions(cor.arbre, err, 1).keys())
                change[message.word] = pro[0]

        return change

    def _handle_grammatical_error(self, sent, message):
        ''' Return a list of changes to do '''
        change = {}
        err = sent[message.start:]
        if message.type == "nbsp":
            change[err] = message.suggestions
        elif message.type == "imp":
            change[err] = message.suggestions
        return change

    def _handle_errors(self, sent):
        changes = {}
        for message in grammalecte_text(sent.text):
            # if type(message) == pygrammalecte.pygrammalecte.GrammalecteSpellingMessage:
            #     correction = _handle_spelling_error(sent.text, message)
            #     changes["orth"].update(correction)
            if type(message) == pygrammalecte.GrammalecteGrammarMessage:
                correction = self._handle_grammatical_error(sent.text, message)
                changes.update(correction)
            
        for w in sent.words:
            for err, remp in changes.items():
                if w.text.find(err) != -1:
                    w.text = w.text.replace(err, remp[0])

        return sent

    def process(self, doc):
        for sent in doc.sentences:
            sent = self._handle_errors(sent)
        return doc

### Correction orthographique à l'aide d'une recherche dans un dictionnaire personnalisé

* dico.py
* mots_fr.py
* slang.py 

La recherche est faite grâce aux fonctions du fichier **correcteur.py** .

In [10]:
def filter_prop(err, props,filter=lambda a,b : len(a) == len(b)):
    ''' Filtre parmis les propositions selon la condition filter '''
    res = {
        "max" : [],
        "normal" : [],
        "min" : []
    }
    for item in props:
        if filter(err, item):
            res["max"].append(item)
        else:
            res["min"].append(item)
        
    return res

def corrector(sms):
    ''' Function corrigeant quelques erreurs avant de passer dans la correction grammalecte '''
    res = None
    erreurs = cor.verification(sms)
    for err in erreurs:
        prop = cor.propositions(cor.arbre, err, 1)
        prop = filter_prop(err, prop)
        if prop["max"] != []:
            res = sms.replace(err, prop["max"][0])
        elif prop["normal"] != []:
            res = sms.replace(err, prop["normal"][0])
        elif prop["min"] != []:
            res = sms.replace(err,prop["min"][0])
    return res

@register_processor("corrector")
class CorrectorProcessor(Processor):
    '''
        Processeur corrector qui agit comme un correcteur orthographique (1er partie de notre correcteur automatique)
        Arborescence et distance Leveinstein
    '''
    _requires = set(["tokenize"])
    _provides = set(["corrector"])

    def __init__(self, config, pipeline, use_gpu):
        pass

    def _set_up_model(self, *args):
        pass

    def _filter_prop(self, err, props,filter=lambda a,b : len(a) == len(b)):
        res = {
            "max" : [],
            "normal" : [],
            "min" : []
        }
        for item in props:
            if filter(err, item):
                res["max"].append(item)
            else:
                res["min"].append(item)
            
        return res

    def _correct_aux(self, sent):
        res = ""
        for w in sent.words:
            erreurs = cor.verification(sent.text)
            for err in erreurs:
                prop = cor.propositions(cor.arbre, err, 1)
                prop = self._filter_prop(err, prop)
                if prop["max"] != []:
                    w.text = w.text.replace(err, prop["max"][0])
                elif prop["normal"] != []:
                    w.text = w.text.replace(err, prop["normal"][0])
                elif prop["min"] != []:
                    w.text = w.text.replace(err,prop["min"][0])
        res += "{}\n".format(sent.text)
        return res

    def process(self, doc):
        for sent in doc.sentences:
            sent = self._correct_aux(sent)

        return doc
    

## Filtre des identifiants

Certains sms comportait à l'origine un prénom, un numéro de téléphone ou bien un adresse. Ces informations ont été cachées en les identifiants de la manière suivantes : $<X\_id>$ .

In [11]:
# Processeur identifier qui remplace chaque balise <X_> par son remplaçant
# ATTENTION : lors du réedit de listePRE il faut penser à le modifier dans _deletePre()
@register_processor("identifier")
class IdentifierProcessor(Processor):
    _requires = set(["tokenize","mwt"])
    _provides = set(["identifier"])

    def __init__(self, config, pipeline, use_gpu):
        pass

    def _set_up_model(self, *args):
        pass
    
    def _deletePre(self, sent):
        listePRE = {
            'PRE_':"prénom",
            'SUR_':"surnom",
            'TEL_':"téléphone",
            'LIE_':"lieu",
            'NOM_':"nom"
        }
        for w in sent.words:
            for pre in listePRE.keys():
                if w.text.find(pre)!=-1:
                    w.text=listePRE[pre]
        return sent
                    
                
    def process(self, doc):
        for sent in doc.sentences:
            sent = self._deletePre(sent)
        return doc

## Passe les chaînes de caractères en lowercase


In [12]:
#Processeur lowercase qui mets en minuscule chacun de nos tokens.
@register_processor("lowercase")
class LowercaseProcessor(Processor):
    _requires = set(["tokenize","mwt","identifier"])
    _provides = set(["lowercase"])

    def __init__(self, config, pipeline, use_gpu):
        pass

    def _set_up_model(self, *args):
        pass

    def process(self, doc):
        doc.text = doc.text.lower()
        for sent in doc.sentences:
            for tok in sent.tokens:
                tok.text = tok.text.lower()

            for word in sent.words:
                word.text = word.text.lower()

        return doc

## Retire les mots inutiles ou ne possèdant pas de poids émotionnel

In [13]:
# Processeur stopword qui enleve les stopword (en utilisant le dictionnaire des stopwords) nltk
# Et qui enleve la ponctuation de nos tokens (ascii de 33 à 64)
@register_processor("stopword")
class StopwordProcessor(Processor):
    ''' Processor that removes all "useless" words '''
    _requires = set(["tokenize","mwt","lowercase","identifier"])
    _provides = set(["stopword"])

    def __init__(self, config, pipeline, use_gpu):
        nltk.download("stopwords")
        self.stopwords = nltk.corpus.stopwords.words("french")
        self.punctuation = [str(chr(i)) for i in range(33, 64)] # ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-']

    def _set_up_model(self, *args):
        pass

    def _stopword_aux(self, sent):
        to_del = []
        for i, w in enumerate(sent.words):
            if w.text in self.stopwords or w.text in self.punctuation: # stopwords from a nltk dictionnary
                to_del.append(i)
            else:
                for c in w.text:
                    if c in self.punctuation: # ascii between 33 and 45
                        # print("in", c)
                        w.text = w.text.replace(c, ' ')

        if (to_del != []):
            for i in to_del[::-1]: # cross the list in reversed order to avoid out of range problems
                if (i < len(sent.words)):
                    del sent.words[i]

        return sent

    def process(self, doc):
        for sent in doc.sentences:
            sent = self._stopword_aux(sent)

        return doc

## Stanza NLP Pipeline

Notre objectif :

1.   Correction automatique
2.   Élément de liste



In [ ]:
# NLP init pipeline
nlp = stanza.Pipeline(lang='fr', processors='tokenize,mwt,identifier,sent_type,stopword,lowercase,feel,lemma', use_gpu=True)

dfSms["corrected_sms"] = np.nan
docs = []
# Pour chaque sms on passe la chaîne de traitement
for i in range(dfSms.shape[0]):
    sms = dfSms.loc[i, "sms"]
    sms_corr = grammalecte_corrector(sms)
    sms_corr = corrector(sms_corr)
    text = sms if sms_corr is None else sms_corr
    dfSms.loc[i, "corrected_sms"] = text
    doc = nlp(text)
    docs.append(doc)
    if i % 100 == 0:        
        print("TEXT({}) : {}".format(i,sms[10:]))
        dfSms.doc = docs
        dfSms.to_csv(FILENAME)

# i = 0
# for sms in dfSms.sms:
#     sms_corr = corrector(sms)
#     text = sms if sms_corr is None else sms_corr
#     doc = nlp(text)

#     docs.append(doc)
#     i += 1

dfSms["doc"] = docs

2021-04-30 09:12:26 INFO: Loading these models for language: fr (French):
| Processor  | Package |
------------------------
| tokenize   | gsd     |
| mwt        | gsd     |
| lemma      | gsd     |
| feel       | default |
| sent_type  | default |
| identifier | default |
| lowercase  | default |
| stopword   | default |

2021-04-30 09:12:26 INFO: Use device: cpu
2021-04-30 09:12:26 INFO: Loading: tokenize
2021-04-30 09:12:26 INFO: Loading: mwt
2021-04-30 09:12:26 INFO: Loading: lemma
2021-04-30 09:12:27 INFO: Loading: feel
2021-04-30 09:12:27 INFO: Loading: sent_type
2021-04-30 09:12:27 INFO: Loading: identifier
2021-04-30 09:12:27 INFO: Loading: lowercase
2021-04-30 09:12:27 INFO: Loading: stopword
2021-04-30 09:12:27 INFO: Done loading processors!


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
TEXT(0) : 
Alors cette rentree?
Va falloir se trouver un p'tit creneau pour se voir!
Dis moi, est-ce que tu sais quand commence les cours de langue non specialiste?
Bisoux
TEXT(100) : ? L'allemand c'est en G001 ! À 9h15 Bisous
TEXT(200) : fk?
TEXT(300) : s j'ai oublié mes souliers chez le prince charmant! Non sérieux le travail m'attend et je suis quelqu'un de sérieux!
TEXT(400) : on ninou
TEXT(500) : mphi b
TEXT(600) : ent ça c passe ?
TEXT(700) :  c'est chiant ! ! Normalement si ça bug il te compte pas !
TEXT(800) : e roméo et juliette à fond
TEXT(900) : piler un peu


# Results

In [ ]:
# doc
print(doc.sent_type)
print(doc.feel_weight)

________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

Ajout des cents mots les plus fréquent dans le corpus sms dans le dico stopwords

In [ ]:
sms10=dfSms.sms.iloc[:10]
sms10

In [ ]:
# # nlp = stanza.Pipeline(lang='fr', processors='tokenize')

# listeFrequence = dict()
# for sms in sms10:
#     doc = nlp(sms)
#     for sentence in doc.sentences:
#         for token in sentence.tokens:
#             if token.text in listeFrequence.keys():
#                 listeFrequence[token.text]+=1
#             else:
#                 listeFrequence[token.text]=1
            
# print(listeFrequence)

In [ ]:
# # Premièrement, on récupère la fréquence totale de chaque mot sur tout le corpus d'artistes
# freq_totale = nltk.Counter()
# for k, v in corpora.iteritems():
#     freq_totale += freq[k]

# # Deuxièmement on décide manière un peu arbitraire du nombre de mots les plus fréquents à supprimer. On pourrait afficher un graphe d'évolution du nombre de mots pour se rendre compte et avoir une meilleure heuristique. 
# most_freq = zip(*freq2.most_common(100))[0]

# # On créé notre set de stopwords final qui cumule ainsi les 100 mots les plus fréquents du corpus ainsi que l'ensemble de stopwords par défaut présent dans la librairie NLTK
# sw = set()
# sw.update(stopwords)
# sw.update(tuple(nltk.corpus.stopwords.words('french')))